In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pmdarima as pm

In [ ]:
train_data = pd.read_csv('train_data.csv')

In [ ]:
test_data = pd.read_csv('test_data.csv')

In [ ]:
#Training
for i in train_data['stock_id'].unique():
    stockdata = train_data[train_data['stock_id']==i]
    arima_model = pm.auto_arima(stockdata['target'],seasonal=True,suppress_warnings=True)
    with open(f'arima_model_{i}', 'wb') as file:
        pickle.dump(arima_model, file)

In [ ]:
#testing Pipeline
true_target = []
predictions = []
test_df = test_data[test_data['seconds_in_bucket']<=480]
test_true_df = test_data[test_data['seconds_in_bucket']>480]
for i in test_df['stock_id'].unique():
    stock_data = test_df[test_df['stock_id']==i]
    stock_data_pred = test_true_df[test_true_df['stock_id']==i]

    # Now, you can load the ARIMA model from the file
    with open(f'arima_model_{i}', 'rb') as file:
        arima_model = pickle.load(file)
    for j in stock_data['date_id'].unique():
        daydata = stock_data[stock_data['date_id']==j]
        day_last_data = stock_data_pred[stock_data_pred['date_id']==j]['target']
        arima_model.update(daydata['target'])
        preds = arima_model.predict(6)
        
        for i in range(len(preds)):
            predictions.append(preds[i])
            true_target.append(day_last_data.values[i])

    


        

In [ ]:
#Absolute Error
absolute_error = np.abs(np.array(predictions[5::6]) - np.array(true_target[5::6]))
#Mean Absolute Error
mae = np.mean(absolute_error)